In [ ]:
import cv2
import respiratory_extraction.utils as utils

In [ ]:
# video_path = '/Volumes/Patrick/Proband06/Logitech HD Pro Webcam C920.avi'
video_path = '/Users/patrick/Desktop/Proband06/Logitech HD Pro Webcam C920.avi'
frames, video_params = utils.read_video_gray(video_path)

In [ ]:
print(video_params)

In [ ]:
import matplotlib.pyplot as plt
import respiratory_extraction.models.baseline as baseline

faces = baseline.face_detection(frames[0])
if len(faces) == 0:
    raise ValueError('No face detected in the first frame')
elif len(faces) > 1:
    raise ValueError('Multiple faces detected in the first frame')

# First face position and size
face1 = faces[0]

# Calculate the region of interest (ROI) based on the face
roi = baseline.calculate_roi_from_face(face1[0], face1[1], face1[2], face1[3])

In [ ]:
# Detect the face in the first frame with Viola Jones algorithm
frame1 = frames[0].copy()

# Draw a rectangle around the face
cv2.rectangle(frame1, (face1[0], face1[1]), (face1[0] + face1[2], face1[1] + face1[3]), (255, 0, 0), 2)

# Draw a rectangle around the chest region
cv2.rectangle(frame1, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (255, 0, 0), 2)

plt.figure(figsize=(5, 5))
plt.imshow(frame1, cmap='gray')
plt.title('Face and chest detection')
plt.show()

## Use the average pixel intensity in the chest region to calculate the breathing rate

In [ ]:
# Calculate the average pixel intensity in the chest region
average_pixels = baseline.average_pixel_intensity(frames, roi)

In [ ]:
import plotly.express as px

# Plot the average pixel intensity in the chest region
fig = px.line(x=range(len(average_pixels)), y=average_pixels,
              title='Average pixel intensity in the chest region')
fig.show()

In [ ]:
# Calculate the fast fourier transform of the average pixel intensity
pixels_fft, freq = baseline.calculate_fft(average_pixels, video_params.fps, min_freq=0.1, max_freq=0.6)

In [ ]:
import numpy as np

# Plot the filtered frequency spectrum
fig = px.line(x=freq, y=np.abs(pixels_fft), title='Filtered Frequency spectrum')
fig.show()

In [ ]:
# Find the frequency with the highest amplitude
max_freq, breathing_rate = baseline.calculate_respiratory_rate(pixels_fft, freq)
max_freq, breathing_rate